<a href="https://colab.research.google.com/github/pangeab-blip/EvGeo-Exercises/blob/main/Oxygen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# OxygenRise — minimal model with infinite reduced-mineral reservoir
# Dynamics:  dx/dt = P_photo - k_total * x
# x(t) = O2 / PAL (dimensionless, 1 PAL ≈ 21% vol); time in Myr.
# P_photo = eps_photo * P0  [PAL/Myr]
# k_total = k_atm + k_fe    [1/Myr]
# Analytic solution: x(t) = x_eq + (x0 - x_eq) * exp(-k_total * t),  x_eq = P_photo / k_total (if k_total>0)

import numpy as np
import math
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Checkbox

def oxygen_simple(
    years_total=3000.0,   # Myr
    dt=0.1,               # Myr
    P0=0.20,              # PAL/Myr (base photosynthetic capacity)
    eps_photo=1.0,        # efficiency [0..2+]
    k_atm=0.02,           # 1/Myr
    k_fe=0.08,            # 1/Myr
    x0=1e-6,              # initial O2 (PAL)
    logy=False            # log-scale for O2 plot
):
    years_total = max(1.0, float(years_total))
    dt = max(1e-4, float(dt))
    t = np.arange(0.0, years_total + dt, dt)

    # Parameters and checks
    P_photo = max(0.0, float(eps_photo) * float(P0))
    k_atm = max(0.0, float(k_atm))
    k_fe  = max(0.0, float(k_fe))
    k_tot = k_atm + k_fe
    x0 = max(0.0, float(x0))

    # Analytic solution
    if k_tot > 0.0:
        x_eq = P_photo / k_tot
        tau = 1.0 / k_tot   # Myr
        x = x_eq + (x0 - x_eq) * np.exp(-k_tot * t)
    else:
        # k_tot = 0: no sink; x grows linearly if P_photo>0
        x_eq = np.inf if P_photo > 0 else x0
        tau = np.inf
        x = x0 + P_photo * t  # PAL

    # Fluxes (PAL/Myr)
    P_series = np.full_like(t, P_photo, dtype=float)
    S_series = k_tot * x

    # ---- Summary ----
    print("=== OxygenRise (infinite reservoir) ===")
    print(f"P_photo = eps_photo * P0 = {eps_photo:.3f} × {P0:.3f} = {P_photo:.3f} PAL/Myr")
    print(f"k_atm = {k_atm:.3f} 1/Myr,  k_fe = {k_fe:.3f} 1/Myr  →  k_total = {k_tot:.3f} 1/Myr")
    print(f"Initial O2: x0 = {x0:.3e} PAL  ({x0*100:.3e}% of PAL)")

    if k_tot > 0.0:
        print(f"Equilibrium: x_eq = P_photo/k_total = {x_eq:.3f} PAL  ({x_eq*100:.3f}% of PAL)")
        print(f"e-folding time: τ = 1/k_total = {tau:.2f} Myr")
    else:
        if P_photo > 0.0:
            print("No sink (k_total=0): O2 grows linearly without bound (x = x0 + P_photo·t).")
        else:
            print("Trivial case: P_photo=0 and k_total=0 → O2 constant.")

    # ---- Plot 1: O2 (% PAL) vs time with equilibrium line/marker ----
    plt.figure(figsize=(7,4))
    y = x * 100.0
    if logy:
        plt.semilogy(t, np.maximum(y, 1e-15))
        plt.ylabel("O$_2$ (% of PAL, log)")
    else:
        plt.plot(t, y)
        plt.ylabel("O$_2$ (% of PAL)")
    plt.xlabel("Time (Myr)")
    title = "O$_2$ (fraction of PAL) — source-sink balance"
    plt.title(title)
    plt.grid(True, linestyle=":")

    # Equilibrium guide
    if k_tot > 0.0 and np.isfinite(x_eq):
        plt.axhline(x_eq*100.0, linestyle="--", linewidth=1)
        plt.text(t[-1]*0.02, (x_eq*100.0)*(1.02 if not logy else 1.0), "equilibrium", fontsize=9)

    # Initial and final markers
    plt.scatter([t[0]], [y[0]], s=30)
    plt.scatter([t[-1]], [y[-1]], s=30)
    plt.show()

    # ---- Plot 2: Fluxes (PAL/Myr) ----
    plt.figure(figsize=(7,4))
    plt.plot(t, P_series, label="P_photo")
    plt.plot(t, S_series, label="Sink = k_total·x")
    plt.xlabel("Time (Myr)")
    plt.ylabel("Flux (PAL/Myr)")
    plt.title("Budget terms: production vs sink")
    plt.legend()
    plt.grid(True, linestyle=":")
    plt.show()

    return {
        "time_Myr": t,
        "O2_PAL": x,
        "x_eq_PAL": (x_eq if k_tot>0 else None),
        "tau_Myr": (tau if k_tot>0 else None),
        "P_photo": P_series,
        "Sink": S_series
    }

interact(
    oxygen_simple,
    years_total=FloatSlider(value=3000.0, min=10.0, max=6000.0, step=10.0, description="Total time (Myr)"),
    dt=FloatSlider(value=0.1, min=0.01, max=2.0, step=0.01, description="dt (Myr)"),
    P0=FloatSlider(value=0.20, min=0.0, max=1.0, step=0.01, description="P0 (PAL/Myr)"),
    eps_photo=FloatSlider(value=1.0, min=0.0, max=2.0, step=0.01, description="ε_photo"),
    k_atm=FloatSlider(value=0.02, min=0.0, max=0.5, step=0.001, description="k_atm (1/Myr)"),
    k_fe=FloatSlider(value=0.08, min=0.0, max=0.5, step=0.001, description="k_fe (1/Myr)"),
    x0=FloatSlider(value=1e-6, min=0.0, max=2.0, step=1e-6, description="O2_0 (PAL)"),
    logy=Checkbox(value=False, description="Log y for O2"),
);

interactive(children=(FloatSlider(value=3000.0, description='Total time (Myr)', max=6000.0, min=10.0, step=10.…